# ECDH Protocol Simulation (Elliptic Curve Diffie-Hellman)

In this notebook, we will simulate secure communication between two IoT devices: a **Sensor (Alice)** and a **Gateway (Bob)**.

We will use the **ECDH** protocol to generate a shared secret key through an insecure channel (such as the internet or radio waves), without ever transmitting the key itself.

### Objectives:
1. Generate public and private keys on elliptic curves.
2. Simulate the exchange of public keys.
3. Calculate the “shared secret” independently.
4. Derive a symmetric key (AES) to encrypt data.

In [ ]:
# Install the standard cryptographic library for Python
!pip install cryptography

In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet
import base64

print("Libraries imported correctly.")

## Step 1: Generating Key Pairs

Each device must generate its own key pair:
* **Private Key:** Must remain SECRET on the device. It is never transmitted.
* **Public Key:** Derived from the private key and can be sent to anyone.

We will use the **SECP256R1** elliptic curve (also known as NIST P-256), a very common standard in IoT.

In [ ]:
# --- DEVICE A (Sensor) ---
# Generates the private key
private_key_A = ec.generate_private_key(ec.SECP256R1())
# Extracts the public key to be sent
public_key_A = private_key_A.public_key()

# --- DEVICE B (Gateway) ---
# Generates the private key
private_key_B = ec.generate_private_key(ec.SECP256R1())
# Extracts the public key to be sent
public_key_B = private_key_B.public_key()

print("✅ Keys generated for both devices.")

### Let's view the public keys
This is the information that travels “in clear” on the network. Even if a hacker intercepts it, they cannot trace it back to the private keys.

In [ ]:
def print_public_key(name, pub_key):
    pem = pub_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    print(f"--- Pubblic key of {name} ---")
    print(pem.decode('utf-8'))

print_public_key("Device A", public_key_A)
print_public_key("Device B", public_key_B)

Step 2: The Exchange (Handshake)

Now the magic of Diffie-Hellman happens.
1. Device A takes B's public key (received from the network) and combines it with its own private key.
2. Device B takes A's public key (received from the network) and combines it with its own private key.

Mathematically: `PrivA * PubB` == `PrivB * PubA`.

In [ ]:
# Device A calculates the shared secret
shared_secret_A = private_key_A.exchange(ec.ECDH(), public_key_B)

# Device B calculates the shared secret
shared_secret_B = private_key_B.exchange(ec.ECDH(), public_key_A)

print(f"Length of the secret calculated by A: {len(shared_secret_A)} bytes")
print(f"Length of the secret calculated by B: {len(shared_secret_B)} bytes")

# Fundamental verification
assert shared_secret_A == shared_secret_B
print("\n✅ SUCCESS: The two secrets are IDENTICAL!")
print(f"Hexadecimal value (first 20 characters): {shared_secret_A.hex()[:20]}...")

Step 3: Key Derivation Function (KDF)

The “shared secret” obtained above is a mathematical point on the curve. It is not yet a good cryptographic key (it may have weak statistical patterns).

We need to pass it through a **KDF (Key Derivation Function)** to obtain a clean symmetric key, for example for the AES algorithm.

In [ ]:
def derive_key(shared_secret):
    # We use standard HKDF with SHA256
    return HKDF(
        algorithm=hashes.SHA256(),
        length=32, # 32 bytes = 256 bits (for AES-256)
        salt=None, # In real protocols, the salt is exchanged during the handshake.
        info=b'iot-handshake', # Application context info
    ).derive(shared_secret)

aes_key_A = derive_key(shared_secret_A)
aes_key_B = derive_key(shared_secret_B)

# We encode in base64 to use it with the Fernet library (which simulates AES).
fernet_key_A = base64.urlsafe_b64encode(aes_key_A)
fernet_key_B = base64.urlsafe_b64encode(aes_key_B)

print(f"Final AES key (Device A): {fernet_key_A}")
print(f"Final AES key (Device B): {fernet_key_B}")

## Step 4: Encrypted Communication

Now that both devices have the same symmetric key, they can exchange encrypted data.
**Device A** sends the temperature. **Device B** receives it and decrypts it.

In [ ]:
# Initialize the symmetric encryption module
cipher_suite_A = Fernet(fernet_key_A)
cipher_suite_B = Fernet(fernet_key_B)

# --- SEND (Device A) ---
cleartext_msg = b"Temperature: 24.5 C"
chipertext_msg = cipher_suite_A.encrypt(cleartext_msg)

print(f"📡 Device A send: {chipertext_msg}")

# ... The message travels over the network ...

# --- RECEPTION (Device B) ---
try:
    plaintext_msg = cipher_suite_B.decrypt(chipertext_msg)
    print(f"🔓 Device B decrypts: {plaintext_msg.decode('utf-8')}")
except Exception as e:
    print("❌ Error in decryption!")

### Conclusion

We have demonstrated how two devices can create a shared secret key using only their public keys, without ever exchanging their private keys or the final secret key.

This is the basis of **DTLS, TLS, HTTPS, and SSH**.